# Instagram Scraper

This notebook is an exploration of a few difficulties in web scraping Instagram.

If you like this example, check out my [`insta_builder.py`](../modules/insta_builder.py) module where I implement the Builder design pattern to create representations of an Instagram profile. This example could be extended to handle profile from other social media platforms by creating a new builder and profile class for each platform.

## Challenges:

1. GET requests to Instagram return executable JavaScript rather than HTML content.
  - To get past this issue I learned how to use Selenium which executes the JavaScript included in the response from Instagram just as a browser would. This gathers and renders the actual HTML content so I can scrape it.
2. Instagram has a soft login-wall to keep clients that are not logged in from viewing the full extent of a profile's public content.
  - When researching how to scroll through a page of unknown length in Selenium, I came across an approach that manipulates the `window` object directly to continue scrolling. Luckily, this direct approach works even after Instagram renders their soft login-wall to the page.
3. Instagram dynamically populates and depopulates images from the browser as you scroll through a profile.
  - To make sure that I capture all of the images as I scroll, I set up an algorithm that takes a 'snapshot' of the current HTML content of the page and then scrolls to the bottom of the page. Scrolling to the bottom of the page causes new content to render and old content to depopulate from the page. It then repeats this process of taking a snapshot and scrolling until it notices that the bottom of the page is no longer extending. I then extract all the image tags from the HTML snapshots and remove duplicates by running them through a set.



In [1]:
import time
from random import randint

from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
# Enter an Instagram profile name as a string below
# And then un-comment the line

# profile_name = ''

In [3]:
instagram_url = f'https://www.instagram.com/{profile_name}'
max_scroll_secs = 300

In [4]:
snapshots = []
def snapshot(browser):
    global snapshots
    snapshots.append(browser.page_source)

In [5]:
# Attribution for scrolling mechanism:
# https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python#27760083

def scroll_and_snapshot(browser, max_scroll_secs):
    SCROLL_PAUSE_TIME = 1

    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")
    
    end_time = time.time() + max_scroll_secs
    
    while time.time() < end_time:
        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height <= last_height:
            snapshot(browser)
            break
        # Create HTML snapshot
        snapshot(browser)
        last_height = new_height

In [6]:
def collect_image_tags(snapshots):
    
    image_tags = []
    
    for html in snapshots:

        soup = BeautifulSoup(html, 'html.parser')
        image_tags += soup.find_all('img', class_='FFVAD')
        
    return set(image_tags)

In [7]:
def capture(snapshots, browser):
        
    image_tags = collect_image_tags(snapshots)

    for index, image in enumerate(image_tags):
        browser.get(image['src'])
        images = browser.find_elements_by_tag_name('img')
        # Edit file name f-string below  as needed
        images[0].screenshot(f'./screenshot_{index}.png')
        wait_time = randint(1, 2)
        time.sleep(wait_time)

In [24]:
# If you need to download/update chromedriver
# https://sites.google.com/a/chromium.org/chromedriver/

browser = webdriver.Chrome('../utilities/chromedriver')
# browser.implicitly_wait(10)
browser.get(instagram_url)

# Find the 'show more' button and click it
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
button = browser.find_element_by_class_name('z4xUb')
button.click()

scroll_and_snapshot(browser, max_scroll_secs)

# ONLY call line below if you want to risk saving a TON of images
# capture(snapshots, browser)

# 2 lines below report # of tags captured w/o saving images
images = collect_image_tags(snapshots)
print(f'Image Links Captured: {len(images)}')

browser.quit()

Image Links Captured: 395
